In [12]:
import pdfplumber
import pandas as pd
import re

In [13]:
with pdfplumber.open("../data-raw/INKOS24.pdf") as pdf:
    lines = [[line['text'] for line in page.extract_text_lines()] for page in pdf.pages]

In [14]:
p2 = lines[1]
p2[:30]

['11/15/2024 - 12:35:07 PM',
 'Election Date: 11/5/2024 Precinct Summary Report',
 'EMS',
 'Kosciusko County, Indiana',
 '2024 General Election',
 'INKOSG24',
 '11/5/2024',
 'E - # Of Election Day 297 PRECINCT STATUS: INCOMPLETE',
 'A - # Of Paper Absentee 15',
 'W - # Of Walk-In Absentee 148',
 'PUBLIC COUNT: 460',
 'VOTER TURNOUT: 71.76%',
 'REGISTERED VOTERS: 641',
 'Precinct ID: 01',
 'Precinct Name: Clay 01',
 '- - - - - - - - - - - - - - - - - - VOTES - - - - - - - - - - - - - - - -',
 'E A W TOTAL %',
 'VOTE FOR 1',
 'Public Question-Const Amendment',
 'VOTES=403',
 '167 7 66 240 59.55% Yes',
 '103 2 58 163 40.45% No',
 'VOTE FOR 1',
 'President and Vice-President of the U.S.',
 'VOTES=458',
 '267 10 117 394 86.03% (R) Trump \\ Vance',
 '25 5 31 61 13.32% (D) Harris \\ Walz',
 '2 0 0 2 0.44% (L) Oliver \\ Ter Maat',
 '1 0 0 1 0.22% (WTP) Kennedy Jr. \\ Shanahan',
 '0 0 0 0 0.00% Write-In']

In [15]:
line = p2[25]
print(line)
#re.search
number = re.search(r'\d%', line)
if number:
    print(number.start())

267 10 117 394 86.03% (R) Trump \ Vance
19


In [16]:
cols = {'Precinct_ID' : [],
        'Precinct_Name' : [],
        'Registered_voters' : [],
        'Vote_on' : [],
        'Voter_subset' : [],
        'Vote_option' : [],
        'Vote_num' : []}
df = pd.DataFrame(data=cols)

In [17]:
for p in lines:
    vote = False #Signifies when a "VOTE FOR ..." line has occurred
    meta = True #Signifies when lines are part of metadata of the precinct
    for (lineno, line) in enumerate(p): #for each line of the page
        if line.find("Precinct ID") > -1: #Find the Precinct ID line
            ID = line.split(':')[-1].strip() #The precinct number is after the ":"
            meta = False #This line signifies we have no more meta data on this page except for the last couple lines
        elif line.find("REGISTERED VOTERS") > -1:
            registered = line.split(':')[-1].strip()
        elif line.find("Precinct Name") > -1: #Find the Precint Name line
            Name = line.split(':')[-1].strip() #The precinct name is after the ":"
        elif line.find('VOTE FOR') > -1: #The 'VOTE FOR..." line means the next line is what is being voted on
            vote = True
        elif vote: #If the 'VOTE FOR..." line occured earlier
            vote = False #Reset this
            on = line #The 'VOTE FOR..." line means the next line is what is being voted on
        elif re.search(r'\d%', line) and not meta: #Lines with percentage numbers that are not metadata are the values we want
            E, A, W, Total, Percent, option = line.split(maxsplit=5) #The line is a row of a table of 6 columns
            addition = pd.DataFrame(data = {'Precinct_ID' : [ID]*5, #The new dataframe to concatenate
                                            'Precinct_Name' : [Name]*5,
                                            'Registered_voters' : [registered]*5,
                                            'Vote_on' : [on]*5,
                                            'Voter_subset' : ['Election Day', 'Paper Absentee', 'Walk-in Absentee', 'Total', 'percentage'],
                                            'Vote_option' : [option]*5,
                                            'Vote_num' : [E, A, W, Total, Percent]})
            df = pd.concat([df,addition], ignore_index=True) #Concatenate the new data to the dataframe

In [18]:
df

,Precinct_ID,Precinct_Name,Registered_voters,Vote_on,Voter_subset,Vote_option,Vote_num
0,01,Clay 01,641,Public Question-Const Amendment,Election Day,Yes,167
1,01,Clay 01,641,Public Question-Const Amendment,Paper Absentee,Yes,7
2,01,Clay 01,641,Public Question-Const Amendment,Walk-in Absentee,Yes,66
3,01,Clay 01,641,Public Question-Const Amendment,Total,Yes,240
4,01,Clay 01,641,Public Question-Const Amendment,percentage,Yes,59.55%
...,...,...,...,...,...,...,...
14175,66,Wayne 13,1,Court of Appeals Dist 4-Pyle III,Election Day,No,0
14176,66,Wayne 13,1,Court of Appeals Dist 4-Pyle III,Paper Absentee,No,0
14177,66,Wayne 13,1,Court of Appeals Dist 4-Pyle III,Walk-in Absentee,No,0
14178,66,Wayne 13,1,Court of Appeals Dist 4-Pyle III,Total,No,0


In [19]:
df.to_csv("../data/INKOS.csv")